In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_partially_arbitrary_generation_politics_rumors_processed.csv
/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_paraphrase_generation_processed.csv
/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_hallucination_processed.csv


In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [4]:
!pip install sentencepiece

In [5]:
import pandas as pd
df = pd.read_csv('/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_paraphrase_generation_processed.csv')

In [6]:
from sklearn.model_selection import train_test_split
train_df, test_df, _,_ = train_test_split(df, df["label"], test_size=0.2, stratify=df['label'],random_state=42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [7]:
def process_data(train_df):
    train_data = []
    prt = "Given a 'passage' determine whether or not it is a piece of misinformation. Only output 'YES' or 'NO'. The 'passage' is: "
    for i in range(len(train_df)):
        temp_text = prt + train_df.iloc[i]['synthetic_misinformation']
        if(train_df.iloc[i]['label']==1):
            temp_label = 'YES'
        else:
            temp_label = 'NO'
        train_data.append({"text": f"[INST] {temp_text} [/INST] {temp_label}"})
    return train_data

In [8]:
train_data = process_data(train_df)
test_data = process_data(test_df)

In [9]:
!pip install datasets -U 

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub>=0.18.0 from https://files.pythonhosted.org/packages/05/09/1945ca6ba3ad8ad6e2872ba682ce8d68c5e63c8e55458ed8ab4885709f1d/huggingface_hub-0.19.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempti

In [10]:
# train_data[:10]

In [11]:
pos_sample = train_df[train_df['label']==1]
neg_sample = train_df[train_df['label']==0]
pos_subset = (pos_sample.sample(n=5))
neg_subset = (neg_sample.sample(n=5))

In [13]:
# pos_subset.head()
temp_df = pd.concat([pos_subset,neg_subset])

In [14]:
# temp_df

In [15]:
train_data = process_data(temp_df)
test_data = process_data(test_df)

In [16]:
from datasets import Dataset
dataset_train = Dataset.from_list(train_data[:])

In [17]:
dataset_train

Dataset({
    features: ['text'],
    num_rows: 10
})

In [18]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [19]:
# The model that you want to train from the Hugging Face hub
model_name = "NousResearch/Llama-2-7b-chat-hf"

# The instruction dataset to use
dataset_name = "mlabonne/guanaco-llama2-1k"

# Fine-tuned model name
new_model = "llama-2-7b-miniguanaco"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [20]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
#     report_to="tensorboard"
)

# Set supervised fine-tuning parameters
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset_train,
#     peft_config=peft_config,
#     dataset_text_field="text",
#     max_seq_length=max_seq_length,
#     tokenizer=tokenizer,
#     args=training_arguments,
#     packing=packing,
# )

# # Train model
# trainer.train()

# # Save trained model
# trainer.model.save_pretrained(new_model)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [24]:
os.rename('/kaggle/working/llama-2-7b-miniguanaco/adapter_config.json', '/kaggle/working/llama-2-7b-miniguanaco/config.json')
os.rename('/kaggle/working/llama-2-7b-miniguanaco/adapter_model.bin', '/kaggle/working/llama-2-7b-miniguanaco/pytorch_model.bin')

In [ ]:
test_df = pd.read_csv('/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_hallucination_processed.csv')

In [21]:
test_df_h = pd.read_csv('/kaggle/input/politifact/synthetic-gpt-3.5-turbo_politifact_partially_arbitrary_generation_politics_rumors_processed.csv')

In [26]:
test_df.head()

,news_id,news_text,label,lang,theme,synthetic_misinformation
362,politifact1180,I am not the first President to take up this c...,0,en,0,I am not the first President to take up this c...
35,politifact15301,This Account has been suspended.Contact your h...,1,en,This Account has been suspended.Contact your h...,Please communicate with your hosting provider ...
27,politifact14643,SHARE this story with your friends!10.3k SHARE...,1,en,SHARE this story with your friends!10.3k SHARE...,Please share this story with your friends! 10....
2,politifact13515,See more of Sid Miller on Facebook,1,en,See more of Sid Miller on Facebook,View additional content featuring Sid Miller b...
186,politifact15418,Former Secretary of State John Kerry may be he...,1,en,Former Secretary of State John Kerry may be he...,"John Kerry, the former Secretary of State, may..."


In [19]:
supervised = []
prompt = "Given a 'passage' determine whether or not it is a piece of misinformation. ONLY output 'YES' or 'NO' with explaination. The 'passage' is: "

pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
n = test_df["synthetic_misinformation"].iloc[2]
print(n)
result = pipe(f"<s> [INST] {prompt}{n} [/INST]")
result[0]['generated_text']


Please share this story with your friends! 10.3k people have already shared it. Recently, Donald Trump suggested that he might end aid to Puerto Rico based on the mayor of San Juan's decision to withhold aid from other parts of the country. The governor of Puerto Rico has appreciated Donald Trump's efforts to help hurricane-affected individuals but the mayor of San Juan has not distributed supplies adequately. The mayor defended her decision by saying that the city does not have enough personnel or resources to distribute supplies nationwide. However, her colleagues didn't accept her explanation and held an emergency meeting to begin impeachment proceedings that will commence on Monday. Donald Trump has provided aid to Puerto Rico and other hurricane-affected areas more effectively than any previous presidents. Puerto Rico needs new leaders who understand and appreciate this fact before they lose the aid that they are not even required to receive.


"<s> [INST] Given a 'passage' determine whether or not it is a piece of misinformation. ONLY output 'YES' or 'NO' with explaination. The 'passage' is: Please share this story with your friends! 10.3k people have already shared it. Recently, Donald Trump suggested that he might end aid to Puerto Rico based on the mayor of San Juan's decision to withhold aid from other parts of the country. The governor of Puerto Rico has appreciated Donald Trump's efforts to help hurricane-affected individuals but the mayor of San Juan has not distributed supplies adequately. The mayor defended her decision by saying that the city does not have enough personnel or resources to distribute supplies nationwide. However, her colleagues didn't accept her explanation and held an emergency meeting to begin impeachment proceedings that will commence on Monday. Donald Trump has provided aid to Puerto Rico and other hurricane-affected areas more effectively than any previous presidents. Puerto Rico needs new lead

In [22]:
supervised = []
prompt = "Given a 'passage' determine whether or not it is a piece of misinformation. ONLY output 'YES' or 'NO'. The 'passage' is: "
count=0
for n in list(test_df_h["synthetic misinformation"]):
#     print(f"{prompt}{n}")

    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=2000)
    result = pipe(f"<s> [INST] {prompt}{n} [/INST]")
    supervised.append(result[0]['generated_text'])
    count+=1
    print(count)


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [23]:
sft_res = []
for x in supervised:
#     sft_res.
    if(x[-3:].strip()=='NO'):
        sft_res.append(0)
    else:
        sft_res.append(1)
    

In [24]:
sft_res

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [ ]:
test_df_h['label'].value_counts()

In [ ]:
zero_shot

In [26]:
from sklearn.metrics import accuracy_score

accuracy_score(test_df_h['label'].values, sft_res)

0.99

In [ ]:
test_df['few_shot'] = sft_res

In [ ]:
test_df.to_csv('test_df3.csv')

In [ ]:
count